<a href="https://colab.research.google.com/github/HisakaKoji/sentence-transformers/blob/master/healsion_delish_combine_recipe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%cd /content
!git clone https://github.com/sonoisa/sentence-transformers
!cd sentence-transformers; pip install -r requirements.txt
!wget -O sonobe-datasets-sentence-transformers-model.tar "https://www.floydhub.com/api/v1/resources/JLTtbaaK5dprnxoJtUbBbi?content=true&download=true&rename=sonobe-datasets-sentence-transformers-model-2"
!tar -xvf sonobe-datasets-sentence-transformers-model.tar

/content
Cloning into 'sentence-transformers'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 707 (delta 2), reused 6 (delta 2), pack-reused 701
Receiving objects: 100% (707/707), 195.58 KiB | 296.00 KiB/s, done.
Resolving deltas: 100% (475/475), done.
     |████████████████████████████████| 450kB 34.3MB/s 
     |████████████████████████████████| 17.1MB 202kB/s 
     |████████████████████████████████| 1.0MB 50.5MB/s 
     |████████████████████████████████| 870kB 46.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=f4fb02b42119ac71ec518a140dcac9f0c3edee4f6293c2fe0173c5f8b4192675
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses
--2020-03-24 03:56:19--  https://www.floydhub.com/api/v1/resources/JLTtbaaK5dprnxoJtUbBbi?content=true&download=true&rename=son

In [6]:
%cd sentence-transformers

[Errno 2] No such file or directory: 'sentence-transformers'
/content/sentence-transformers


In [0]:
%tensorflow_version 2.x
from sentence_transformers import SentenceTransformer
import numpy as np

model_path = "/content/training_bert_japanese"
model = SentenceTransformer(model_path, show_progress_bar=False)

In [0]:
import pandas as pd

In [0]:
df = pd.read_csv('all_delish.csv')

In [0]:
df.head

In [0]:
df['description'] = df['店舗名'] + ' ' + df['description'] # + ' ' + df['step']

In [0]:
df

In [0]:
df = df.rename(columns={'店舗名': 'name'})

In [0]:
df = df.dropna()

In [0]:
df = df[~df.duplicated(subset='url')]

In [0]:
df

In [0]:
sentences = df['description'].values.tolist()

In [0]:
sentence_vectors = model.encode(sentences)

In [178]:
sentence_vectors[0][0]

-0.28582418

In [0]:
import numpy as np

In [0]:
data1 = np.empty(768)

In [0]:
data1 = sentence_vectors[0]

In [0]:
data1

In [0]:
data1 = np.vstack((data1, sentence_vectors[2] ) )

In [0]:
df_vector = pd.Series(sentence_vectors)

In [168]:
df_vector[0].shape

(768,)

In [0]:
sentences_name = df['name'].values.tolist()
sentences_url = df['url'].values.tolist()

In [0]:
from sklearn.cluster import KMeans

num_clusters = 8
clustering_model = KMeans(n_clusters=num_clusters)
clustering_model.fit(sentence_vectors)
cluster_assignment = clustering_model.labels_

clustered_sentences = [[] for i in range(num_clusters)]
for sentence_id, cluster_id in enumerate(cluster_assignment):
    clustered_sentences[cluster_id].append(sentences_name[sentence_id][:10])

for i, cluster in enumerate(clustered_sentences):
    print("Cluster ", i+1)
    print(cluster)
    print("")

In [0]:
df_queries = pd.read_csv('cookpad_keyword.txt',header = None)
#df_queries = pd.read_csv('healsio_recipe.csv',header = None)

In [0]:
df = pd.read_csv('healsio_recipe.csv')
df['description'] = df['name'] # + df['ingredient'] + df['step']
queries =  df['description'].values.tolist()
sentences_names = df['name'].values.tolist()
sentences_urls = df['url'].values.tolist()


In [0]:
data1 = np.ndarray([])
import scipy.spatial

#queries = ['ゆきぽよ','美術館', 'グルメ', '究極の料理','スポーツ大会', '子供がはしゃげる','東広島市　料理','東広島市　おでかけ','古墳','アニメ','細菌','ビッグバン','甲賀忍者']
#queries = df_queries[0].values.tolist()
query_embeddings = model.encode(queries)

closest_n = 5

for i,(name,url,query, query_embedding) in enumerate(zip(sentences_names,sentences_urls,queries, query_embeddings)):
    distances = scipy.spatial.distance.cdist([query_embedding], sentence_vectors, metric="cosine")[0]

    results = zip(range(len(distances)), distances)
    results = sorted(results, key=lambda x: x[1])

    print("\n\n======================\n\n")
    print("Query:", name + ' ' + url)
    print("\nTop 5 most similar sentences in corpus:")

    for j,(idx, distance) in enumerate(results[0:closest_n]):
        print(sentences_name[idx].strip(), sentences_url[idx].strip(),"(Score: %.4f)" % (distance / 2),idx)
        if j == 0 and i == 0:
          data1 = sentence_vectors[idx]
          print(idx)
        elif j == 0:
          data1 = np.vstack((data1, sentence_vectors[idx] ) )
          print(idx)


In [0]:
df_h = pd.read_csv('healsio_recipe.csv')

In [0]:

df_h['description'] = df_h['name'] #  +  df['step']#+ df['ingredient'] 

In [0]:
df_h = df_h[~df_h.duplicated(subset='url')]

In [0]:
sentences =  df_h['description'].values.tolist()
sentences_name = df_h['name'].values.tolist()
sentences_url = df_h['url'].values.tolist()

In [0]:
sentence_vectors = model.encode(sentences)



In [0]:
#df_queries = pd.read_csv('cookpad_keyword.txt',header = None)
df_queries = pd.read_csv('kurashiru_keyword.txt',header = None)


In [0]:
df_out = pd.read_csv('output.txt')

In [0]:
import scipy.spatial

queries = ['ゆきぽよ','美術館', 'グルメ', '究極の料理','スポーツ大会', '子供がはしゃげる','東広島市　料理','東広島市　おでかけ','古墳','アニメ','細菌','ビッグバン','甲賀忍者']
queries = df_queries[0].values.tolist()
query_embeddings = model.encode(queries)

closest_n = 5
adata = pd.DataFrame()
for query, query_embedding in zip(queries, query_embeddings):
    distances = scipy.spatial.distance.cdist([query_embedding], data1, metric="cosine")[0]

    results = zip(range(len(distances)), distances)
    results = sorted(results, key=lambda x: x[1])

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop 5 most similar sentences in corpus:")
    i = 0
    for idx, distance in results[0:closest_n]:
        i+=1
        print(sentences_name[idx].strip(), sentences_url[idx].strip(),"(Score: %.4f)" % (distance / 2))
        temp = pd.DataFrame({'query': query,
                   'name':sentences_name[idx].strip(),'rank': i,
                   'url': sentences_url})
        adata = pd.concat([adata,temp], axis=0)

In [0]:
%load_ext tensorboard
import os
logs_base_dir = "runs"
os.makedirs(logs_base_dir, exist_ok=True)

In [0]:
import torch
from torch.utils.tensorboard import SummaryWriter

import tensorflow as tf
import tensorboard as tb
tf.io.gfile = tb.compat.tensorflow_stub.io.gfile

summary_writer = SummaryWriter()
summary_writer.add_embedding(mat=np.array(sentence_vectors), metadata=sentences_name)

In [0]:
%tensorboard --logdir {logs_base_dir}

In [0]:
!kill 1265